In [4]:
##IMPORTS##
import chess
import numpy as np
import pandas as pd

df = pd.read_csv('chessData.csv')

def gen_legal_moves(board,frac=1):
    legal_moves = board.legal_moves
    legal_moves = [str(move) for move in legal_moves]
    if frac == 1:
        return legal_moves
    else:
        return np.random.choice(legal_moves, math.floor(frac*len(legal_moves)), replace=False)

In [38]:
##SYNTETIC DATA GENERATION##
def generate_synthetic_data_dict(N,size,df,seed=1):
    if size == 1:
        return generate_synthetic_data_single(N,seed,df)
    data = []
    np.random.seed(seed)
    for i in range(N):
        dict = {}
        random_list = np.random.multinomial(size, np.ones(size)/size, size=1)[0]
        board = chess.Board(df['FEN'][i])
        legal_moves = gen_legal_moves(board)
        for j in range(size):
            if random_list[j] != 0:
                dict[np.random.choice(legal_moves)] = random_list[j]
        data.append(dict)

    return data

def generate_synthetic_data_single(N,seed,df):
    data = []
    np.random.seed(seed)
    for i in range(N):
        board = chess.Board(df['FEN'][i])
        legal_moves = gen_legal_moves(board)
        data.append(np.random.choice(legal_moves))
    return data

path=("stockfish-windows-x86-64\stockfish\stockfish-windows-x86-64.exe")


def evaluate_move(board, move, path):
    #Evaluates the move before move and after move return the difference
    board1 = board.copy()
    board1.push(chess.Move.from_uci(move))
    board2 = board.copy()
    #Return differnce of board1 and board2


def amount_legal_moves_taken(l , df):
    amount = 0
    for i in range(len(l)):
        board = chess.Board(df['FEN'][i])
        legal_moves = gen_legal_moves(board)
        if l[i] in legal_moves:
            amount += 1
    return amount

def amount_index_zero(l,df):
    amount = 0
    for i in range(len(l)):
        board = chess.Board(df['FEN'][i])
        legal_moves = gen_legal_moves(board)
        if l[i] == legal_moves[0]:
            amount += 1
    return amount

def amount_mean_taken_over_mu_moves():
    pass


amount = amount_index_zero(generate_synthetic_data_dict(10,1,df,1),df)
amount 

1

In [100]:
import chess
import chess.engine
import pandas as pd
import numpy as np

def evaluate_move(fen,move):
    # Path to Stockfish executable
    stockfish_path = "stockfish-windows-x86-64\stockfish\stockfish-windows-x86-64.exe"
    
    with chess.engine.SimpleEngine.popen_uci(stockfish_path) as engine:
        engine.configure({"Threads": 1})  # Configure Stockfish to run in single-threaded mode for consistency
        # Create a chess board from FEN
        board = chess.Board(fen)
        # Determine which player is to move
        is_white_to_move = board.turn == chess.WHITE
        # Evaluate the initial position
        initial_info = engine.analyse(board, chess.engine.Limit(time=1.0))  # Increased time limit for stability
        initial_score = initial_info["score"].white().score(mate_score=10000)  # Get the POV score for White
        
        # If it's Black to move, invert the score
        if not is_white_to_move:
            initial_score = -initial_score
        
        board = chess.Board(fen)
        board.push(chess.Move.from_uci(move))
        info = engine.analyse(board, chess.engine.Limit(time=0.1))  # Increased time limit for stability
        move_score_uci = info["score"].white().score(mate_score=10000)  # Get the POV score for White
        # If it's Black to move, invert the score
        if not is_white_to_move:
            move_score_uci = -move_score_uci

        move_difference = move_score_uci - initial_score
        return move_difference

def evaluate_moves(fen, move_uci, dataframe):
     # Path to Stockfish executable
    stockfish_path = "stockfish-windows-x86-64\stockfish\stockfish-windows-x86-64.exe"
    # Load Stockfish engine
    with chess.engine.SimpleEngine.popen_uci(stockfish_path) as engine:
        engine.configure({"Threads": 1})  # Configure Stockfish to run in single-threaded mode for consistency
        
        # Create a chess board from FEN
        board = chess.Board(fen)
        
        # Determine which player is to move
        is_white_to_move = board.turn == chess.WHITE
        
        # Evaluate the initial position
        initial_info = engine.analyse(board, chess.engine.Limit(time=1.0))  # Increased time limit for stability
        initial_score = initial_info["score"].white().score(mate_score=10000)  # Get the POV score for White
        
        # If it's Black to move, invert the score
        if not is_white_to_move:
            initial_score = -initial_score
        
        # Generate all legal moves
        legal_moves = list(board.legal_moves)

        board = chess.Board(fen)
        board.push(chess.Move.from_uci(move_uci))
        info = engine.analyse(board, chess.engine.Limit(time=0.1))  # Increased time limit for stability
        move_score_uci = info["score"].white().score(mate_score=10000)  # Get the POV score for White
        # If it's Black to move, invert the score
        if not is_white_to_move:
            move_score_uci = -move_score_uci
        move_uci_difference = move_score_uci - initial_score
        
        # Evaluate all legal moves
        move_differences = []
        for move in legal_moves:
            board = chess.Board(fen)
            board.push(move)
            info = engine.analyse(board, chess.engine.Limit(time=0.1))  # Increased time limit for stability
            move_score = info["score"].white().score(mate_score=10000)  # Get the POV score for White
            # If it's Black to move, invert the score
            if not is_white_to_move:
                move_score = -move_score
            difference = move_score - initial_score
            move_differences.append((move.uci(), difference))
            board.pop()
        
        # Convert to DataFrame
        moves_df = pd.DataFrame(move_differences, columns=['move', 'score_difference'])
        moves_df.head()

       
        return move_uci_difference,initial_score, moves_df
        

def percentiles(moves_df):
        score_75th_percentile = np.percentile(moves_df['score_difference'], 75)
        score_25th_percentile = np.percentile(moves_df['score_difference'], 25)
        score_50th_percentile = np.percentile(moves_df['score_difference'], 50)
        # Return results
        result = {
            "score_75th_percentile": score_75th_percentile,
            "score_50th_percentile": score_50th_percentile,
            "score_25th_percentile": score_25th_percentile,}
        return result

def in_percentile(score,percentile):
    
    if score >= percentile["score_75th_percentile"]:
        return np.array([0,0,0,1])
    elif percentile["score_50th_percentile"] <= score < percentile["score_75th_percentile"]:
        return np.array([0,0,1,0])
    elif percentile["score_25th_percentile"] <= score < percentile["score_50th_percentile"]:
        return np.array([0,1,0,0])
    else:
        return np.array([1,0,0,0])
    
def percentile_distribution(moves,df):
    result = np.zeros(4)
    for i in range(len(moves)):
        move_difference, inital_score, moves_df = evaluate_moves(df['FEN'][i], moves[i], df)
        result += in_percentile(move_difference,percentiles(moves_df))
    return result
                                


In [101]:
#Example use single move
fen = df['FEN'][0]
move = generate_synthetic_data_single(1,1,df)[0]
score = evaluate_move(fen,move)
score

-37

In [76]:
l = generate_synthetic_data_dict(1000,1,df,1)
for i in range(len(l)):
    print(f"{i}: " , evaluate_moves(df['FEN'][i],l[i],df)["provided_move_score_difference"])

0:  -40
1:  -61
2:  -109
3:  -227
4:  -153
5:  -134
6:  -411
7:  -8
8:  -94
9:  -90
10:  -14
11:  -149
12:  -5
13:  -456
14:  -65
15:  -108
16:  -73
17:  -3
18:  -449
19:  -438
20:  -55
21:  -593
22:  -109
23:  -458
24:  -284
25:  -9
26:  -374
27:  -518
28:  -336
29:  -678
30:  -542
31:  -351
32:  -203
33:  -346
34:  -53
35:  -887
36:  -181
37:  -508
38:  -283
39:  -641
40:  -291
41:  -1009
42:  -192
43:  -610
44:  -175
45:  -959
46:  -49
47:  -10281
48:  -471
49:  -646
50:  -225
51:  49
52:  -226
53:  -10274
54:  -210
55:  -218
56:  -20
57:  -887
58:  -9437
59:  -9743
60:  -2
61:  -9565
62:  -2
63:  -475
64:  -176
65:  -66
66:  -613
67:  -184
68:  -497
69:  -2
70:  -156
71:  -71
72:  -232
73:  -18
74:  -260
75:  -541
76:  -340
77:  -49
78:  -79
79:  -463
80:  -52
81:  -26
82:  -354
83:  -450
84:  -485
85:  -523
86:  -439
87:  -259
88:  -203
89:  -47
90:  -621
91:  -454
92:  -111


KeyboardInterrupt: 

In [65]:
##DATA LOADING##

In [ ]:
##DATA PREPROCESSING##

In [ ]:
##DATA ANALYSIS##